# 数据关联与合并

在大多应用中，数据是分布存储在不同的文件或数据表中。Pandas 提供多个方法来关联与合并多个数据：
- 使用`pd.concat()`与`df.append()`把数据沿轴方向进行关联或合并；
- 使用`pd.merge()`等函数根据列把多个数据帧进行关联合并

本节通过一些例子来介绍数据的关联与合并操作。

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 数据集关联

在关系型数据库中，经常会通过一个键或多个键来合并两个数据集，SQL 的连接操作有如下：
- 内连接（INNER JOIN）
- 左连接（LEFT JOIN）
- 右连接（RIGHT JOIN）
- 全连接（FULL JOIN）

在 Pandas 中，通过数据集的共有键来关联两个数据集时，也有对应的关联方式：
- 内连接（inner），对两个数据集共有键的交集进行关联，即只保留匹配的行.
- 左外连接（left），对左边数据集所有键进行关联，即只保留左边全部行，右边没有匹配字段取空值。
- 右外连接（right），对右边数据集所有键进行关联，即只保留右边全部行，左边没有匹配字段取空值。
- 外连接（outer），也成为全连接。对两个数据集共有键的并集进行关联，即保留左右两边全部行，没有被匹配的字段取空值。

四种关联方式可以参见如下示意图：
![关联方向](../images/pandas_join_types.jpg)

Numpy 数组对象可以沿指定轴向进行关联，也称为合并、拼接、堆叠等。对于 `Series` 与 `DataFrame` 对象来说，也存在类似应用场景。然而不同的是，二者都存在行或列索引。在进行关联时，需要考虑更多问题，例如索引之间有重复如何堆叠，或索引不相同如何拼接。

## 沿轴向关联

Pandas 提供`pd.concat()`函数来实现数据关联，其使用语法为：
```python
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True)
```
主要参数：
- `obj`，需要合并的 Pandas 对象列表或字典。
- `axis=0`，指定数据进行合并的轴向，缺省沿行的方向。
- `join='outer'`，指定关联方式
- `join_axes=None`，用于指定其它轴的特定索引。
- `keys=None`，
- `level=None`，指定多层索引的层级。
- `ingore_index=False`，不保留轴上的索引，产生新的索引对象。

下面分别给出`Series`与`DataFrame`的合并示例：

### `Series`对象关联

创建三个索引不重复的`Series`对象：

In [4]:
# index 没有重复的情况
s1 = pd.Series(np.random.randn(2), index=range(0,2))
s2 = pd.Series(np.random.randn(2), index=range(2,4))
s3 = pd.Series(np.random.randn(2), index=range(4,6))

s1
s2
s3

0   -0.770138
1   -0.485088
dtype: float64

2    2.572241
3   -0.770608
dtype: float64

4    2.547966
5    0.403677
dtype: float64

把这三个对象组成一个列表，然后传入到`pd.concat()`函数：

In [7]:
# 沿行合并
pd.concat([s1, s2, s3])

0   -0.388592
1    0.875285
2   -1.251216
3    0.023215
4    0.419322
5    1.369025
dtype: float64

下面来看看索引存在重复的情况，其合并结果如何：

In [5]:
# index 有重复的情况
s1 = pd.Series(np.random.randn(2), index=range(0, 2))
s2 = pd.Series(np.random.randn(2), index=range(1, 3))
s3 = pd.Series(np.random.randn(3), index=range(1, 4))
s1
s2
s3

0   -1.209441
1   -0.323585
dtype: float64

1    1.646882
2    1.680360
dtype: float64

1   -1.717757
2    0.465847
3   -0.361156
dtype: float64

In [6]:
# 沿行合并
pd.concat([s1, s2, s3])

0   -1.209441
1   -0.323585
1    1.646882
2    1.680360
1   -1.717757
2    0.465847
3   -0.361156
dtype: float64

当索引有重复时，会保留重复结果。传入`ignore_index=True`参数，则不再保留旧的索引：

In [14]:
# 沿行合并
pd.concat([s1, s2, s3], ignore_index=True)

0   -1.130292
1   -0.313944
2    0.004703
3    0.651472
4    0.033454
5    1.035251
6   -0.425681
dtype: float64

传入参数`axis=1`，将不同对象的列关联起来，当行索引重复异或不重复时，使用不同关联方式效果会有所不同：

In [16]:
# 沿列进行内连接
pd.concat([s1, s2, s3], axis=1, join='inner')

,0,1,2
1,-0.313944,0.004703,0.033454


In [17]:
# 沿列进行外连接
pd.concat([s1, s2, s3], axis=1, join='outer')

,0,1,2
0,-1.130292,NaN,NaN
1,-0.313944,0.004703,0.033454
2,NaN,0.651472,1.035251
3,NaN,NaN,-0.425681


### `DataFrame`对象关联

对于 `DataFrame` 对象，指定参数 `axis=0`，会沿着行方向进行合并，使用列名进行数据关联；指定 `axis=1`，则会沿列方向进行合并，使用行索引进行关联：

In [7]:
df1 = pd.DataFrame(np.random.randint(0, 10, (3, 2)), index=['a', 'b', 'c'], columns=['A', 'B'])
df2 = pd.DataFrame(np.random.randint(0, 10, (2, 2)), index=['a', 'b'], columns=['B', 'D'])

df1
df2

,A,B
a,3,2
b,4,3
c,4,3


,B,D
a,2,2
b,3,0


In [24]:
# 沿行方向关联，使用外连接
pd.concat([df1, df2], axis=0, join='outer', sort=False)

,A,B,D
a,4.0,0,NaN
b,5.0,2,NaN
c,8.0,4,NaN
a,NaN,8,8.0
b,NaN,8,7.0


In [25]:
# 沿行方向关联，使用内连接
pd.concat([df1, df2], axis=0, join='inner', sort=False)

,B
a,0
b,2
c,4
a,8
b,8


In [26]:
# 沿列方向关联，使用外连接
pd.concat([df1, df2], axis=1, join='outer', sort=False)

,A,B,B,D
a,4,0,8.0,8.0
b,5,2,8.0,7.0
c,8,4,NaN,NaN


In [29]:
# 沿列方向关联，使用内连接
pd.concat([df1, df2], axis=1, join='inner', ignore_index=True, sort=False)

,0,1,2,3
a,4,0,8,8
b,5,2,8,7


### `df.append()`

`DataFrame`对象方法`df.append()`只是`pd.concat()`的简化版本，即把两个对象进行沿着行方向（`axis=0`)，使用`join='outer'`进行外连接关联，其使用语法为：
```python
df.append(other, ignore_index=False, verify_integrity=False, sort=None)
```

In [38]:
# 合并两个对象
df1.append(df2, sort=False)

,A,B,D
a,4.0,0,NaN
b,5.0,2,NaN
c,8.0,4,NaN
a,NaN,8,8.0
b,NaN,8,7.0


## 按列进行关联

Pandas 函数`pd.merge()`根据单个或多个列将不同 `DataFrame` 关联起来，生成新的对象，其使用语法为：
```python
pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
```
- `left, right`，参与合并的两个`DataFrame`对象
- `how='inner'`，关联方式，缺省为内连接（inner）
- `on=None`，用于连接的列名，必须存在于两个`DataFrame`对象中。缺省使用二者的交集作为连接键。
- `left_on`，左数据集中用作连接键的列。
- `right_on`，右数据集中用作连接键的列。
- `left_index`，使用左数据集的行索引作为连接键。
- `right_index`，使用右数据集的行索引作为连接键。
- `sort=False`，通过连接键按字母顺序对合并数据进行排序。
- `suffixes=('_x', '_y')`，在重叠情况下，添加到列名后的字符串元组。

下面创建两个`DataFrame`对象：

In [24]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data' : np.random.randint(0, 10, 7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data' : np.random.randint(0, 10, 3)},
                   index=['c','b','a'])

In [26]:
df1
df2

,key,data
0,b,4
1,b,7
2,a,9
3,c,0
4,a,5
5,a,0
6,b,8


,key,data
c,a,2
b,b,8
a,d,5


这两个对象的行索引是不同，列包括`key`与`data`。下面语句使用列`key`来进行数据关联，关联方式缺省为内连接：

In [27]:
pd.merge(df1, df2, on='key')

,key,data_x,data_y
0,b,4,8
1,b,7,8
2,b,8,8
3,a,9,2
4,a,5,2
5,a,0,2


可以看出，结果只保留二者有交集的的行，传递的索引对象会被丢弃。由于`df1`的列 `key`中存在重复值，故而最终结果时多对多连接结果，即二者行行的笛卡儿积。下面进行左连接，同时指定列名重叠时添加到列名后的字符串元组：

In [30]:
# 重复列名
pd.merge(df1, df2, on='key', how='left', suffixes=('_left', '_right'))

,key,data_left,data_right
0,b,4,8.0
1,b,7,8.0
2,a,9,2.0
3,c,0,NaN
4,a,5,2.0
5,a,0,2.0
6,b,8,8.0


如果左右数据集的关联键（列名）不一致，可以使用`left_on`, `right_on`来分别指定。下面首先更改列名：

In [38]:
df1 = df1.rename(columns={'key':'key1'})
df2 = df2.rename(columns={'key':'key2'})

然后进行指定关联键，使用右连接进行关联：

In [39]:
pd.merge(df1, df2, left_on='key1', right_on='key2', how='right')

,key1,data_x,key2,data_y
0,b,4.0,b,8
1,b,7.0,b,8
2,b,8.0,b,8
3,a,9.0,a,2
4,a,5.0,a,2
5,a,0.0,a,2
6,NaN,NaN,d,5


有时候，其中一个`DataFrame`的关联键是它的索引，可以使用`left_index`或`right_index`来指定。下面语句中，`df1`的关联键使用`key`列，`df2`的关联键则使用索引，关联方式使用外连接：

In [40]:
# 某一边按索引连接
pd.merge(df1, df2, left_on='key1', right_index=True)

,key1,data_x,key2,data_y
0,b,4,b,8
1,b,7,b,8
6,b,8,b,8
2,a,9,d,5
4,a,5,d,5
5,a,0,d,5
3,c,0,a,2


`DataFrame`对象提供有方法`df.merge()`，其功能与`pd.merge()`函数类似。还有一个方法`df.join()`，实际上是`pd.merge()`函数的简化版。

In [46]:
df1.join(df2, on='key1', lsuffix='_x', sort=True)

,key1,data_x,key2,data
2,a,9,d,5
4,a,5,d,5
5,a,0,d,5
0,b,4,b,8
1,b,7,b,8
6,b,8,b,8
3,c,0,a,2
